# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Our outcome of interest in this problem is a binary variable indicating whether a student is likely to pass or fail. Hence the problem at hand is one of classification given that the outcome is categorical. With the help of the features/ student attributes, our aim to accurately predict whether a student will be 'classified' as likely to 'pass' or likely to 'fail'.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [127]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [128]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1 ## minus 1 for excluding the response variable 

# TODO: Calculate passing students
n_passed = sum(student_data.passed == "yes")

# TODO: Calculate failing students
n_failed = sum(student_data.passed == "no")

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/ n_students) * 100 

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [129]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

In [130]:
## Checking the data types of all features and outcome
X_all.dtypes

school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
dtype: object

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [131]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [132]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
    
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=300, random_state=42)
X_train_test = X_train.reindex(range(300))
X_test.reindex(range(95))

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

***Logistic regression: ***
Logistic regression can be used for fault detection in industrial printing. The features used in this application are sensor readings from various machine parts.

Since the outcome of ogistic regression is log odds of the outcome, we can evaluate and interpret the probabilities of the outcomes (on the basis of which classification is done). Additionally, because of the model's simplicity, the coefficients of the features can also be interpreted easily to gain additional insight.

Logistic regression however assumes that the relationship between the features and the log odds of the binary outcome is linear. Hence it is not a good choice in situations where that is not the case.

I believe the simplicity and interpretability of the model makes it a good candidate for the problem of detecting likelihood of student failure. The coefficients obtained by fitting the model for the deographic and educational features of students can be used in designing targeted interventions. Moreover, I also believe the regularization feature of logistic regression can be used for reducing variance/ overfitting and improving prediction performance on test student data. 

***Adaboost: *** 
One of the real life applications of Adaboost is face detection in images and video.

Adaboost does not make any linearity assumptions as in case of Logistic Regression. The model can also handle high dimensional problems very well. On the down side however, because of being computationally intensive, the model may be slow to fit on larger and high dimensional datasets. Moreover, the model is not very interpretable since the ensemble consists of numerous adaptively grown trees.

However, as an ensemble method that learns slowly, adaboost generally avoids overfitting and gives superior predictive performance on test data on most datasets. Hence, we can ignore one of its major weaknesses i.e. computational inefficiency (which is not very important for our problem since our dataset is not too large).
*** SVMs: ***
SVMs are used in bio-medical laboratories to identify cancer types given patients gene chip data. 

SVMs can also perform well on spaces that are not linearly separable (by using the kernel trick). Additionally, SVM models can be regulrized using tuning parameters to give optimal performance. And finally, since SVM can be defined as a complex optimization problem, it finds a global minimum. On the downside however, the algorithm becomes increasingly slower as the size of the data set increases. SVM may not be a best choice in problems where different classes are nor clearly separable.  

Even though logistic regression is a simple, interpretable and useful choice given our model, the linearity assumption between the features and the log odds is possibly not true in case of our dataset. Given this limitation, SVM is a valid alternative to test on our dataset. Morover, SVMs can also be regularized via tunable parameters to avoid overfitting on training student data. 

Lastly, given our dataset is not too large, poor scalability of SVM is not a major concern for the problem at hand.  
 
** References: **
1. http://ieeexplore.ieee.org/document/245054/?arnumber=245054&tag=1
2. http://ieeexplore.ieee.org/document/6496751/?arnumber=6496751&tag=1
3. https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms
4. http://www.stat.ucla.edu/~sczhu/Courses/UCLA/Stat_231/Face_detection.html
5. http://www3.ntu.edu.sg/home/elpwang/pdf_web/05_svm_basic.pdf

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [133]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    
    if y_train.values.dtype != 'float64':
        return f1_score(target.values, y_pred, pos_label='yes')
    else:
        return f1_score(target.values, y_pred, pos_label=1)

def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [134]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

from numpy.random import seed, choice

# TODO: Initialize the three models
clf_A = LogisticRegression()
clf_B = AdaBoostClassifier()
clf_C = SVC()

# TODO: Set up the training set sizes
seed(500)
index_100 = choice(X_train.index, 100)
X_train_100 = X_train.loc[index_100, :]
y_train_100 = y_train[index_100]

index_200 = choice(X_train.index, 200)
X_train_200 = X_train.loc[index_200, :]
y_train_200 = y_train[index_200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for i in ['A', 'B', 'C']:
    train_predict(locals()["clf_"+str(i)], X_train_100, y_train_100, X_test, y_test)
    train_predict(locals()["clf_"+str(i)], X_train_200, y_train_200, X_test, y_test)
    train_predict(locals()["clf_"+str(i)], X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0161 seconds
Made predictions in 0.0034 seconds.
F1 score for training set: 0.9272.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7442.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0023 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8168.
Made predictions in 0.0018 seconds.
F1 score for test set: 0.7761.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0044 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8468.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8060.
Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.1719 seconds
Made predictions in 0.0106 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0102 seconds.
F1 score for test set: 0.6984.
Training a AdaBoostClassifier using a training s

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.002           |        0.0012          |      0.9272      | 0.7442          |
| 200               |         0.0042          |        0.0006          |      0.8168      | 0.7761          |
| 300               |         0.0053          |        0.0006          |      0.8468      | 0.8060          |

** Classifer 2 - AdaBoost**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.1612              |       0.0127           |     1.000        |    0.6984       |
| 200               |     0.1497              |       0.0077           |     0.8939       |    0.8637       |
| 300               |     0.1522              |       0.0097           |     0.8637       |    0.7820       |

** Classifer 3 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.002            |     0.0009             |   0.8889         |    0.7712       |
| 200               |        0.0035           |     0.0013             |   0.9091         |    0.8000       |
| 300               |        0.0097           |     0.0021             |   0.8761         |    0.7838       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
I have chosen Logistic Regression because:

1. We can evaluate probabilities of failure of a students with the help of the model. Such iformation can help us assess the urgency or the intensity of intervention necessary from the school. The probabilities can also help the school administration in dividing at risk students into categories and create targteted programs for students in each category. 
2. The logistic regression model can also be regularized using 'l1' or 'l2' penalty to avoid overfitting and improve prediction performance on test data. 
3. I think the interpretability of the logistic regression model is a huge strength as we can study the coefficients associated with the features and examine which ones are very predictive of student failure. Such information can also help the school to intervene effectively.
4. Assessing the results of the experiments in the previous section also show that logistic regression trains and predicts relatively much faster as compared to my other two choices. Although the f1 scores are obtained by a simple validation approach, they also seem to indicate a superior performance by logistic regression.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
The school's goal is to predict whether or not a student is likely to fail given some data so that a timely intervention can be made. The variable of interest in the problem at hand is a binary outcome ('yes' if a student fails and 'no' otherwise). Logistic Regression can be thought of as a mathemetical model that maps a set of input variables (student characteristics in the dataset) to a probability of the outcome (binary variable of interest). The evaluated probability is used to classify or predict the binary outcome of interest (student failure in our case). Initially we need student data  to train a model that can later generalize well and predict failure for any student whose data has not been used to train the model. Therefore we need to take measures and make sure that the model can not only explain the data reasonably well (not perfectly) so that it can also generalize and predict failure for any unseen student input variables.    

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [135]:
## turning 'yes'/'no' labels to numeric labels
y_train = pd.get_dummies(y_train).yes
y_test = pd.get_dummies(y_test).yes

In [136]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.cross_validation import ShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {'C': np.arange(0.05,1.05,0.05), 'penalty': ['l1', 'l2'] }

# TODO: Initialize the classifier
clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method/ 10-fold cross validation
cv = ShuffleSplit(X_train.shape[0], n_iter=100, test_size=0.1, random_state=0)

grid_obj = GridSearchCV(clf, parameters, f1_scorer, cv = cv)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0004 seconds.
Tuned model has a training F1 score of 0.8330.
Made predictions in 0.0003 seconds.
Tuned model has a testing F1 score of 0.8028.


In [138]:
## Check the parameters of optimal classifier
clf

LogisticRegression(C=0.20000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The f1 score of the optimal model obtained using the GridCV appraoch is 0.8028. It is slightly lesser 0.8060, the test f1 score obtained by a simple validation approach (The parameter settings can be seen in the cell above). Initially, I was confused as to why the optimal f1 score is lesser than the f1 score for a model with default parameter settings. But later, I realized that the disparity can be explained by the fact that the former has been evaluated by using the 10-fold cross validation wheras the latter was just obtained by a simple validation approach (training set size = 300, test set size = 95). 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.